In [1]:
import numpy as np
import pandas as pd
import vectorbt as bt
import yfinance as yf
import pandas_ta as ta

In [2]:
btc = yf.download('BTC-USD', start='2024-12-01', end='2024-12-07', interval='5m')
btc.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
btc = btc.asfreq('5min')
btc

[*********************100%***********************]  1 of 1 completed


ValueError: Length mismatch: Expected axis has 5 elements, new values have 6 elements

In [3]:
# Initial parameters
initial_capital = 100000
risk_per_trade = 0.02  # 2%
stop_loss = 0.18  # 18%
take_profit = 0.40  # 40%

In [4]:
type(btc['Close'])

pandas.core.series.Series

In [5]:
btc['rsi']= ta.rsi(btc['Close'], length=14)
btc['ema_short'] = ta.ema(btc['Close'], length=8)
btc['ema_long'] = ta.ema(btc['Close'], length=18)

btc['long_entry'] = ((btc['ema_short'] > btc['ema_long']) & (btc['rsi'] < 40)).astype(int)
# btc['long_exit'] = ((btc['rsi']>70)).astype(int)

btc['short_entry'] = ((btc['ema_long'] > btc['ema_short']) & (btc['rsi'] > 70)).astype(int)
# btc['short_exit'] = (btc['rsi']<30).astype(int)

# btc['long_entry'] = ((btc['rsi'] < 15)).astype(int)
# btc['long_exit'] = ((btc['rsi']>70)).astype(int)

# btc['short_entry'] = ((btc['rsi'] > 85)).astype(int)
# btc['short_exit'] = (btc['rsi']<30).astype(int)


(btc)

,Adj Close,Close,High,Low,Open,Volume,rsi,ema_short,ema_long,long_entry,short_entry
Datetime,,,,,,,,,,,
2024-12-01 00:00:00+00:00,96523.296875,96523.296875,96523.296875,96411.148438,96475.828125,0,NaN,NaN,NaN,0,0
2024-12-01 00:05:00+00:00,96572.632812,96572.632812,96572.632812,96544.164062,96567.765625,0,NaN,NaN,NaN,0,0
2024-12-01 00:10:00+00:00,96603.750000,96603.750000,96603.750000,96586.500000,96586.500000,0,NaN,NaN,NaN,0,0
2024-12-01 00:15:00+00:00,96578.320312,96578.320312,96628.117188,96578.320312,96611.117188,0,NaN,NaN,NaN,0,0
2024-12-01 00:20:00+00:00,96593.789062,96593.789062,96634.906250,96589.710938,96589.710938,0,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-12-06 23:35:00+00:00,99918.835938,99918.835938,99918.835938,99802.367188,99802.367188,0,31.648852,100083.474619,100370.680712,0,0
2024-12-06 23:40:00+00:00,100011.796875,100011.796875,100011.796875,99913.085938,99913.085938,0,35.207069,100067.546231,100332.903466,0,0
2024-12-06 23:45:00+00:00,100026.937500,100026.937500,100026.937500,99958.406250,100003.578125,0,35.793335,100058.522069,100300.696522,0,0


In [6]:
position_size = 0.02

In [7]:
pf = bt.Portfolio.from_signals(btc['Close'], entries=btc['long_entry'], short_entries=btc['short_entry'],fees=0.001, slippage=0.001, sl_stop=0.04, sl_trail=True, upon_stop_exit='reverse')

In [8]:
returns = pf.total_return()
print(returns)

-0.056166842315514456


In [9]:
pf.stats()

Start                         2024-12-01 00:00:00+00:00
End                           2024-12-06 23:55:00+00:00
Period                                  6 days 00:00:00
Start Value                                       100.0
End Value                                     94.383316
Total Return [%]                              -5.616684
Benchmark Return [%]                           3.498111
Max Gross Exposure [%]                            100.0
Total Fees Paid                                0.482087
Max Drawdown [%]                                6.54634
Max Drawdown Duration                   1 days 08:40:00
Total Trades                                          3
Total Closed Trades                                   2
Total Open Trades                                     1
Open Trade PnL                                 0.271317
Win Rate [%]                                        0.0
Best Trade [%]                                -1.601583
Worst Trade [%]                               -4

In [10]:
pf.plot(
    subplots=['trades', 'orders', 'trade_pnl', 'cum_returns']
    )


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '614508e2-6fc2-4384-8c6f-7c1750092037',
              'x': array([datetime.datetime(2024, 12, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 1, 0, 10, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 6, 23, 45, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 6, 23, 50, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 6, 23, 55, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([ 96523.296875 ,  96572.6328125,  96603.75     , ...